In [19]:
from __future__ import print_function

import astropy as apy

from astropy.coordinates import SkyCoord
from astropy import units as u
import numpy as np
import pylab as plt
from astropy.table import Table
from astropy.io import fits
import csv

In [10]:
def filter_stars(table):
    star_type_mask = table['MEAN_OBJECT_TYPE'] > 5
    return table[star_type_mask]
    
def filter_galaxies(table):
    star_type_mask = table['MEAN_OBJECT_TYPE'] == 3
    return table[star_type_mask]

def filter_pm(table):
    dec_pm_min = np.abs(table["DEC_PM_CLIP"]) < 0.04
    dec_pm_max = np.abs(table["DEC_PM_CLIP"]) >= 0
    ra_pm_min = np.abs(table["RA_PM_CLIP"]) < 0.04
    ra_pm_max = np.abs(table["RA_PM_CLIP"]) >= 0
    return table[dec_pm_min & dec_pm_max & ra_pm_max & ra_pm_max]
    

def read_hlc_file(filename = "HLC.RA_02_to_03.fits", objfilter="stars", extra=False, absval=True):
    f = fits.open(filename)
    table = Table(f[1].data)
    
    if objfilter == "stars":
        table = filter_stars(table)
        #table = filter_pm(table)
    elif objfilter == "galaxies":
        table = filter_galaxies(table)
    elif objfilter == "off" or objfilter is None:
        pass
    else:
        raise ValueError("Unknown option for objfilter: {0}".format(objfilter))

    #get interesting values
    mags = Table(table['MEAN_PSFMAG_CLIP'], names=('u', 'g', 'r', 'i','z')) 
    if absval:
        rapm = np.abs(table['RA_PM_CLIP'].data)
        decpm = np.abs(table['DEC_PM_CLIP'].data)
    else:
        rapm = table['RA_PM_CLIP'].data
        decpm = table['DEC_PM_CLIP'].data
    if extra:
        ra = table["RA_MEAN_CLIP"].data
        dec = table["RA_MEAN_CLIP"].data
        expmags = Table(table['MEAN_EXPMAG_CLIP'], names=('u', 'g', 'r', 'i','z')) 
        return mags, rapm, decpm, ra, dec, expmags
    
    return mags, rapm, decpm    

In [11]:
def _arrayify(a):

    """Turn seq into a 1-d numpy array."""

    try:
        arr = np.array(a)
    except:
        raise

    return arr


def xmatch(ra1,dec1,ra2,dec2,maxdist=None,units='deg',method='astropy',**kwargs):

    """Cross-match two sets of ra & dec coordinates locally (i.e. all coordinates are in RAM).

    The function will search for counterparts of ra1/dec1 coordinates
    in the in ra2/dec2 coordinate set, i.e. one can consider ra2/dec2
    to be the catalog that will be searched.

    Parameters
    ----------
    ra1, dec1: 1-d array-like sequences
        RA and declination of first coordinate set, in units of `units`

    ra2, dec2: 1-d array-like sequences
        RA and declination of second coordinate set, in units of `units`

    maxdist : float or None
        If not `None`, then it is the maximum angular distance (in
        units of `units`) to be considered. All distances greater than
        that will be considered non-matches. If `None`, then all
        ra1/dec1 will have matches in ra2/dec2.

    units : str
        Units of `ra1`, `dec1`, `ra2`, `dec2`. Default: 'deg' (decimal degrees).

    method : str
        Currently only astropy's :func:`match_to_catalog_sky()` method
        is supported, i.e. the default 'astropy'.

    Other Parameters
    ----------------
    nthneighbor : int, optional
        If ``method='astropy'``. Which closest neighbor to search for.
        Typically ``1`` is desired here, as that is correct for
        matching one set of coordinates to another. The next likely
        use case is ``2``, for matching a coordinate catalog against
        *itself* (``1`` is inappropriate because each point will find
        itself as the closest match).

    Returns
    -------
    idx : 1-d array
        Index values of the ra1/dec1 counterparts found in
        ra2/dec2. Thus ra2[idx], dec2[idx] will select from the
        ra2/dec2 catalog the matched counterparts of the ra1/dec1
        coordinate pairs.

        If `maxdist` was not `None` but a number instead, then 'idx'
        only contains the objects matched up to the `maxdist` radius.

    dist2d : 1-d array
        The angular distances of the matches found in the ra2/dec2
        catalog. In units of `units`.

        If `maxdist` was not `None` but a number instead, then
        'dist2d' only contains the objects matched up to the `maxdist`
        radius.

    """

    # turn coordinate sequences into 1d arrays
    ra1 = _arrayify(ra1)
    dec1 = _arrayify(dec1)
    ra2 = _arrayify(ra2)
    dec2 = _arrayify(dec2)

    if method == 'astropy':
        unit = getattr(u,units)
        c1 = SkyCoord(ra=ra1*unit, dec=dec1*unit)
        c2 = SkyCoord(ra=ra2*unit, dec=dec2*unit)
        idx, dist2d, dist3d = c1.match_to_catalog_sky(c2,**kwargs)

        if maxdist is not None:
            sel = (dist2d <= maxdist*unit)
            idx = idx[sel]
            dist2d = dist2d[sel]

    elif method == 'q3cpy':  # serialize Adam's code in Python first
        raise NotImplementedError("Method '%s' not yet implemented." % method)

    else:
        raise Exception("'%s' is not a valid method." % method)

    return idx, dist2d



In [13]:
mags, rapm, decpm, ra1, dec1, expmags = read_hlc_file("../homeworks/Hw1_DinoBektesevic/HLC.RA_00_to_01.fits", objfilter=None, extra=True, absval=False)

In [26]:
t = apy.io.ascii.read("stripe82_315_ra_45_-1_3_dec_0.txt")
t   

ra,dec,pmra,pmraerr,pmdec,pmdecerr,mjd,deltamjd,gmag,grms,gerr,rmag,rrms,rerr,imag,irms,ierr,class_star,fwhm,ebv,nphot
float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,int64
315.012148225,-1.29859422822,1275.32477859,1314.59016223,-91.9294582075,1398.31935707,57598.6972853,10.879461,99.989998,999999.0,9.9899998,20.612076,999999.0,0.027147001,99.989998,999999.0,9.9899998,0.97888899,1.380337,0.087261997,2
315.010753856,-1.29788009321,-714.960501254,3414.340885,-272.622408939,3447.44418474,57598.6972853,10.879461,99.989998,999999.0,9.9899998,21.986279,999999.0,0.079820998,99.989998,999999.0,9.9899998,0.899216,1.522221,0.087286003,2
315.013867349,-1.29628741391,-2339.88941393,6293.68251465,-2186.86337026,6311.70266805,57598.6972853,10.879461,99.989998,999999.0,9.9899998,22.269653,999999.0,0.095987,99.989998,999999.0,9.9899998,0.34095901,2.4877191,0.087237,2
315.003075442,-1.29364556559,106.992177304,118.045727223,18.3782817287,119.195840746,57717.185732,360.90506,22.649281,999999.0,0.092128001,21.409351,999999.0,0.052671,99.989998,999999.0,9.9899998,0.866193,1.79105,0.08743,3
315.003150251,-1.29117939779,-1.56484703704,24.1412972143,-6.21648569305,29.3903214509,57717.185732,360.90506,18.699413,999999.0,0.0066479999,18.291037,999999.0,0.0064750002,99.989998,999999.0,9.9899998,0.98412299,1.3741961,0.087435998,3
315.006782513,-1.29351168006,-29.658981189,25.0975481946,-10.1823353055,30.1658241086,57717.185732,360.90506,19.521944,999999.0,0.0093940003,19.056189,999999.0,0.0096169999,99.989998,999999.0,9.9899998,0.98353201,1.371506,0.087362997,3
315.000335946,-1.28725232795,74.9851549866,53.7685288431,59.4659526729,56.2419562016,57717.185732,360.90506,21.79801,999999.0,0.044064,20.41136,999999.0,0.023931,99.989998,999999.0,9.9899998,0.97916597,1.506238,0.087499999,3
315.005266004,-1.2860947224,4.54835673519,33.2638368645,-19.4873175145,33.946344009,57761.4979698,360.90506,21.709343,999999.0,0.046,20.621595,999999.0,0.028348999,99.989998,999999.0,9.9899998,0.974819,1.406352,0.087398998,4
315.007057874,-1.28901810752,2.84316614519,50.7506305808,28.1143447746,53.3655914347,57717.185732,360.90506,21.587086,999999.0,0.037985999,20.32089,999999.0,0.023398999,99.989998,999999.0,9.9899998,0.98008198,1.560813,0.087362997,3


In [27]:
ra2 = np.asarray(t["ra"])
dec2 = np.asarray(t["dec"])

In [28]:
idx, dist2d = xmatch(ra1, dec1, ra2, dec2, maxdist=0.000277778)

In [30]:
idx

array([], dtype=int64)